In [1]:
import torch
import math
torch.cuda.empty_cache()
from recbole.quick_start import load_data_and_model

In [2]:
# RepeatNet
# config_rn, model_rn, dataset_rn, train_data_rn, valid_data_rn, test_data_rn = load_data_and_model("saved/RepeatNet-Jun-06-2022_13-20-15.pth")
# SARSRec MFS + CLR
# config_ours, model_ours, dataset_ours, train_data_ours, valid_data_ours, test_data_ours = load_data_and_model("/work/hawshiuancha_umass_edu/projects/RecBole-1/saved/SASRec-Jul-21-2022_03-28-09.pth")

# RepeatNet
config_rn, model_rn, dataset_rn, train_data_rn, valid_data_rn, test_data_rn = load_data_and_model("/work/hawshiuancha_umass_edu/projects/RecBole-1/saved/RepeatNet-Jul-23-2023_22-25-20.pth")

# Softmax + Mi
config_smi, model_smi, dataset_smi, train_data_smi, valid_data_smi, test_data_smi = load_data_and_model("/work/hawshiuancha_umass_edu/projects/RecBole-1/saved/GRU4Rec_Ours-Jul-23-2023_20-33-59.pth")

# Softmax + CPR:100 + Mi
config_scmi, model_scmi, dataset_scmi, train_data_scmi, valida_data_scmi, test_data_scmi = load_data_and_model("/work/hawshiuancha_umass_edu/projects/RecBole-1/saved/GRU4Rec_Ours-Jul-23-2023_20-32-45.pth")

05 Aug 19:51    INFO  [Training]: train_batch_size = [128] negative sampling: [None]
05 Aug 19:51    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}]
05 Aug 19:53    INFO  [Training]: train_batch_size = [128] negative sampling: [None]
05 Aug 19:53    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}]
05 Aug 19:55    INFO  [Training]: train_batch_size = [128] negative sampling: [None]
05 Aug 19:55    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}]


In [3]:
from recbole.utils import get_trainer

In [4]:
trainer_rn = get_trainer(config_rn['MODEL_TYPE'], config_rn['model'])(config_rn, model_rn)
trainer_smi = get_trainer(config_smi['MODEL_TYPE'], config_smi['model'])(config_smi, model_smi)
trainer_scmi = get_trainer(config_scmi['MODEL_TYPE'], config_scmi['model'])(config_scmi, model_scmi)

In [5]:
results_rn = trainer_rn.evaluationWithInteractions(test_data_rn, load_best_model=False, model_file="/work/hawshiuancha_umass_edu/projects/RecBole-1/saved/RepeatNet-Jul-23-2023_22-25-20.pth")
results_smi = trainer_smi.evaluationWithInteractions(test_data_smi, load_best_model=False, model_file="/work/hawshiuancha_umass_edu/projects/RecBole-1/saved/GRU4Rec_Ours-Jul-23-2023_20-33-59.pth")
results_scmi = trainer_scmi.evaluationWithInteractions(test_data_scmi, load_best_model=False, model_file="/work/hawshiuancha_umass_edu/projects/RecBole-1/saved/GRU4Rec_Ours-Jul-23-2023_20-32-45.pth")

In [6]:
result_rn, interactions_rn_array, scores_rn_array = results_rn
result_smi, interactions_smi_array, scores_smi_array = results_smi
result_scmi, interactions_scmi_array, scores_scmi_array = results_scmi

In [7]:
result_rn

{'recall@10': 0.2092,
 'mrr@10': 0.1488,
 'ndcg@10': 0.1628,
 'hit@10': 0.2092,
 'precision@10': 0.0209}

In [8]:
result_smi

{'recall@10': 0.0246,
 'mrr@10': 0.0085,
 'ndcg@10': 0.0122,
 'hit@10': 0.0246,
 'precision@10': 0.0025}

In [9]:
result_scmi

{'recall@10': 0.0355,
 'mrr@10': 0.0168,
 'ndcg@10': 0.0212,
 'hit@10': 0.0355,
 'precision@10': 0.0036}

In [10]:
# input_data_rn, result_rn, interactions_rn, scores_rn, positive_u_rn, positive_i_rn = results_rn
# input_data_smi, result_smi, interactions_smi, scores_smi, positive_u_smi, positive_i_smi = results_smi
# input_data_scmi, result_scmi, interactions_scmi, scores_scmi, positive_u_scmi, positive_i_scmi = results_scmi

In [11]:
def get_tokens_info(dataset_name):
    file_path = "dataset/" + dataset_name + "/" + dataset_name + ".item"
    f = open(file_path, "r")
    lines = f.readlines()
    token_to_info_header = ["app", "dev", "early", "genres", "meta", "price", "publisher", "sentiment", "specs", "tags"]
    token_to_info = {}
    for line in lines[1:]:
        try:
            arr = line.split('\t')
            item_token = arr[4]
            item_info = arr[:4] + arr[5:7] + arr[8:]
            token_to_info[item_token] = item_info
        except e:
            print("Incorrect data format!", line)
    return token_to_info

In [12]:
token_to_info = get_tokens_info("steam")

In [13]:
print("Number of batches:", len(interactions_rn_array))
print(interactions_rn_array[0])

Number of batches: 82
The batch_size of interaction: 4096
    user_id, torch.Size([4096]), cpu, torch.int64
    play_hours, torch.Size([4096]), cpu, torch.float32
    product_id, torch.Size([4096]), cpu, torch.int64
    timestamp, torch.Size([4096]), cpu, torch.float32
    item_length, torch.Size([4096]), cpu, torch.int64
    play_hours_list, torch.Size([4096, 50]), cpu, torch.float64
    product_id_list, torch.Size([4096, 50]), cpu, torch.int64
    timestamp_list, torch.Size([4096, 50]), cpu, torch.float64




In [14]:
from prettytable import PrettyTable
from scipy.special import softmax
import numpy as np
def visualize_top_k_suggestions(scores, token_to_info, id_to_token_map, k=5):
    probs = softmax(scores.cpu().numpy())
    top_k_ids = np.flip(probs.argsort())[:k]
    results = PrettyTable(["s/no.", "token", "app", "dev", "early", "genres", "meta", "price", "publisher", "sentiment", "specs", "tags", "probability"])
    results._max_width = {"s/no." : 4, "token" : 4, "app": 4, "dev": 4, "early": 4, "genres": 5, "meta": 3, "price": 4, "publisher": 4, "sentiment": 4, "specs": 5, "tags":5}
    for i in range(k):
        identifier = top_k_ids[i]
        token = id_to_token_map[identifier]
        info = token_to_info[token]
        row = ['N/A']*13
        row[0] = identifier
        row[1] = token
        if len(info) >= 1:
            row[2] = info[0]
        if len(info) >= 2:
            row[3] = info[1]
        if len(info) >= 3:
            row[4] = info[2]
        if len(info) >= 4:
            row[5] = info[3]
        if len(info) >= 5:
            row[6] = info[4]
        if len(info) >= 6:
            row[7] = info[5]
        if len(info) >= 7:
            row[8] = info[6]
        if len(info) >= 8:
            row[9] = info[7]
        if len(info) >= 9:
            row[10] = info[8]
        if len(info) >= 10:
            row[11] = info[9]
        row[12] = probs[top_k_ids[i]]
        results.add_row(row)
    print(results)

In [15]:
def visualize_input(identifier, token_to_info, id_to_token_map):
    results = PrettyTable(["s/no.", "token", "app", "dev", "early_access", "genres", "metascore", "price", "publisher", "sentiment", "specs", "tags"])
    results._max_width = {"s/no." : 4, "token" : 4, "app": 4, "dev": 4, "early": 4, "genres": 5, "meta": 3, "price": 4, "publisher": 4, "sentiment": 4, "specs": 5, "tags":5}
    token = id_to_token_map[identifier]
    info = token_to_info[token]
    row = ['N/A']*12
    row[0] = identifier
    row[1] = token
    if len(info) >= 1:
            row[2] = info[0]
    if len(info) >= 2:
        row[3] = info[1]
    if len(info) >= 3:
        row[4] = info[2]
    if len(info) >= 4:
        row[5] = info[3]
    if len(info) >= 5:
        row[6] = info[4]
    if len(info) >= 6:
        row[7] = info[5]
    if len(info) >= 7:
        row[8] = info[6]
    if len(info) >= 8:
        row[9] = info[7]
    if len(info) >= 9:
        row[10] = info[8]
    if len(info) >= 10:
        row[11] = info[9]
    results.add_row(row)
    print(results)

In [16]:
# RepeatNet results
repeat_selection_losses_rn = []
nonrepeat_selection_losses_rn = []
for interactions_rn, scores_rn in zip(interactions_rn_array, scores_rn_array):
    for i in range(len(interactions_rn['product_id'])):
        if interactions_rn['product_id'][i] in interactions_rn['product_id_list'][i]:
            repeat_selection_losses_rn.append(-math.log(scores_rn[i][int(interactions_rn['product_id'][i])]))
        else:
            nonrepeat_selection_losses_rn.append(-math.log(scores_rn[i][int(interactions_rn['product_id'][i])]))
print("RepeatNet losses:")
print(f"Average loss when actual item repeated: {np.mean(repeat_selection_losses_rn)}")
print(f"Average loss when actual item not repeated: {np.mean(nonrepeat_selection_losses_rn)}")

RepeatNet losses:
Average loss when actual item repeated: 1.433872209640739
Average loss when actual item not repeated: 7.477564935989621


In [17]:
# Softmax + Mi results
repeat_selection_losses_smi = []
nonrepeat_selection_losses_smi = []
for interactions_smi, scores_smi in zip(interactions_smi_array, scores_smi_array):
    for i in range(len(interactions_smi['product_id'])):
        if interactions_smi['product_id'][i] in interactions_smi['product_id_list'][i]:
            repeat_selection_losses_smi.append(-math.log(scores_smi[i][int(interactions_smi['product_id'][i])]))
        else:
            nonrepeat_selection_losses_smi.append(-math.log(scores_smi[i][int(interactions_smi['product_id'][i])]))
print("Softmax + Mi losses:")
print(f"Average loss when actual item repeated: {np.mean(repeat_selection_losses_smi)}")
print(f"Average loss when actual item not repeated: {np.mean(nonrepeat_selection_losses_smi)}")

Softmax + Mi losses:
Average loss when actual item repeated: 8.274012296898263
Average loss when actual item not repeated: 10.102628799457856


In [18]:
# Softmax + CPR:100 + Mi results
repeat_selection_losses_scmi = []
nonrepeat_selection_losses_scmi = []
for interactions_scmi, scores_scmi in zip(interactions_scmi_array, scores_scmi_array):
    for i in range(len(interactions_scmi['product_id'])):
        if interactions_scmi['product_id'][i] in interactions_scmi['product_id_list'][i]:
            repeat_selection_losses_scmi.append(-math.log(scores_scmi[i][int(interactions_scmi['product_id'][i])]))
        else:
            nonrepeat_selection_losses_scmi.append(-math.log(scores_scmi[i][int(interactions_scmi['product_id'][i])]))
print("Softmax + CPR:100 + Mi losses:")
print(f"Average loss when actual item repeated: {np.mean(repeat_selection_losses_scmi)}")
print(f"Average loss when actual item not repeated: {np.mean(nonrepeat_selection_losses_scmi)}")

Softmax + CPR:100 + Mi losses:
Average loss when actual item repeated: 7.1270048418774
Average loss when actual item not repeated: 9.967234585455143


In [19]:
num_batches = len(interactions_rn_array)

In [20]:
# # In this cell, we select examples where there are only 5 items in the history and Ground Truth is in history
# num_batches = len(interactions_rn_array)
# example_indexes = [] # this is a list of tuples to retrieve example
# for batch_index in range(num_batches):
#     interactions_rn = interactions_rn_array[batch_index]
#     for i in range(len(interactions_rn)):
#         if (interactions_rn['product_id_list'][i] != 0).sum().item() <= 5 and (interactions_rn['product_id_list'][i] == interactions_rn['product_id'][i].item()).sum().item() > 0:
#             example_indexes.append((batch_index, i))

In [29]:
# In this cell, we select examples where there are only 5 items in the history
num_batches = len(interactions_rn_array)
example_indexes = [] # this is a list of tuples to retrieve example
for batch_index in range(num_batches):
    interactions_rn = interactions_rn_array[batch_index]
    for i in range(len(interactions_rn)):
        if (interactions_rn['product_id_list'][i] != 0).sum().item() <= 5:
            example_indexes.append((batch_index, i))

In [30]:
len(example_indexes)

128852

In [22]:
# in this cell, we select final examples where Softmax + CPR:100 + Mi has the GT in top 3 
# but SoftMax + Mi, and Repeatnet don't 
# final_example_indexes = []
# for i, j in example_indexes:
#     interactions_rn = interactions_rn_array[i]
#     scores_rn = scores_rn_array[i][j]
#     scores_smi = scores_smi_array[i][j]
#     scores_scmi = scores_scmi_array[i][j]
#     sorted_scores_rn = scores_rn.sort(dim=0, descending=True).indices
#     sorted_scores_smi = scores_smi.sort(dim=0, descending=True).indices
#     sorted_scores_scmi = scores_scmi.sort(dim=0, descending=True).indices
#     if (interactions_rn['product_id'][j] in sorted_scores_scmi[:3]) and (interactions_rn['product_id'][j] not in sorted_scores_smi[:3]) and (interactions_rn['product_id'][j] not in sorted_scores_rn[:3]):
#         final_example_indexes.append((i, j))

In [31]:
def hasIntersection(list1, list2):
    list3 = [value for value in list1 if value in list2]
    return len(list3) > 0

In [34]:
# in this cell, we select final examples where Softmax + CPR:100 + Mi has the GT in top 3 
# and RepeatNet and SoftMax + Mi have some games from history in top 3 items
final_example_indexes = []
for i, j in example_indexes:
    interactions_rn = interactions_rn_array[i]
    scores_rn = scores_rn_array[i][j]
    scores_smi = scores_smi_array[i][j]
    scores_scmi = scores_scmi_array[i][j]
    sorted_scores_rn = scores_rn.sort(dim=0, descending=True).indices
    sorted_scores_smi = scores_smi.sort(dim=0, descending=True).indices
    sorted_scores_scmi = scores_scmi.sort(dim=0, descending=True).indices
    if (interactions_rn['product_id'][j] in sorted_scores_scmi[:3]) and (interactions_rn['product_id'][j] not in sorted_scores_smi[:3]) and (interactions_rn['product_id'][j] not in sorted_scores_rn[:3]):
        if hasIntersection(interactions_rn['product_id_list'][j], sorted_scores_rn[:3]) and hasIntersection(interactions_rn['product_id_list'][j], sorted_scores_smi[:3]):
            final_example_indexes.append((i, j))

In [35]:
print(len(final_example_indexes))

21


In [36]:
for i, j in final_example_indexes:
    interactions_rn = interactions_rn_array[i]
    print("\n------History------\n")
    id_to_token_map = dataset_rn.field2id_token['product_id']
    for k in range(50):
        token = id_to_token_map[interactions_rn['product_id_list'][j][k]]
        if token == '[PAD]':
            break
        info = token_to_info[token]
        app_name = info[0]
        print(app_name)
    rn_top_3_predictions = scores_rn_array[i][j].sort(dim=0, descending=True)
    rn_top3_predictions_indices = rn_top_3_predictions.indices[:3]
    rn_top3_predictions_scores = rn_top_3_predictions.values[:3]
    smi_top_3_predictions = scores_smi_array[i][j].sort(dim=0, descending=True)
    smi_top3_predictions_indices = smi_top_3_predictions.indices[:3]
    smi_top3_predictions_scores = smi_top_3_predictions.values[:3]
    scmi_top3_predictions = scores_scmi_array[i][j].sort(dim=0, descending=True)
    scmi_top3_predictions_indices = scmi_top3_predictions.indices[:3]
    scmi_top3_predictions_scores = scmi_top3_predictions.values[:3]
    print("\n---Ground truth----")
    print(token_to_info[id_to_token_map[interactions_rn['product_id'][j]]][0])
    print("\n---RepeatNet top 3 suggestions---")
    for k in range(3):
        print(token_to_info[id_to_token_map[rn_top3_predictions_indices[k]]][0], rn_top3_predictions_scores[k].item())
    print("\n---Softmax + Mi top 3 suggestions---")
    for k in range(3):
        print(token_to_info[id_to_token_map[smi_top3_predictions_indices[k]]][0], smi_top3_predictions_scores[k].item())
    print("\n---Softmax + CPR:100 + Mi top 3 suggestions---")
    for k in range(3):
        print(token_to_info[id_to_token_map[scmi_top3_predictions_indices[k]]][0], scmi_top3_predictions_scores[k].item())    
    print("\n---Ground truth scores---")
    print("\nRepeatNet: ", scores_rn_array[i][j][interactions_rn['product_id'][j]].item())
    print("Softmax + Mi: ", scores_smi_array[i][j][interactions_rn['product_id'][j]].item())
    print("Softmax + CPR:100 + Mi: ", scores_scmi_array[i][j][interactions_rn['product_id'][j]].item())


------History------

The Elder Scrolls IV: Oblivion\xae Game of the Year Edition
Reus
Home
Rust

---Ground truth----
Chivalry: Medieval Warfare

---RepeatNet top 3 suggestions---
Rust 0.4181603193283081
Starbound 0.007947315461933613
Borderlands 2 0.007651458960026503

---Softmax + Mi top 3 suggestions---
Rust 0.037949319928884506
No More Room in Hell 0.031017225235700607
Portal 2 0.025028159841895103

---Softmax + CPR:100 + Mi top 3 suggestions---
Chivalry: Medieval Warfare 0.07268843054771423
Mount & Blade: Warband 0.028470205143094063
Saints Row: The Third 0.02274739183485508

---Ground truth scores---

RepeatNet:  0.004932015668600798
Softmax + Mi:  0.005092207342386246
Softmax + CPR:100 + Mi:  0.07268843054771423

------History------

Delver
Rust
The Ship: Murder Party
STAR WARS\u2122 - The Force Unleashed\u2122 Ultimate Sith Edition

---Ground truth----
Mount & Blade: Warband

---RepeatNet top 3 suggestions---
STAR WARS\u2122 - The Force Unleashed\u2122 Ultimate Sith Edition 0.0

In [ ]:
def visualize_inputs(identifiers, token_to_info, id_to_token_map):
    results = PrettyTable(["s/no.", "token", "app", "dev", "early_access", "genres", "metascore", "price", "publisher", "sentiment", "specs", "tags"])
    results._max_width = {"s/no." : 4, "token" : 4, "app": 4, "dev": 4, "early": 4, "genres": 5, "meta": 3, "price": 4, "publisher": 4, "sentiment": 4, "specs": 5, "tags":5}
    for identifier in identifiers:
        token = id_to_token_map[identifier]
        info = token_to_info[token]
        row = ['N/A']*12
        row[0] = identifier
        row[1] = token
        if len(info) >= 1:
                row[2] = info[0]
        if len(info) >= 2:
            row[3] = info[1]
        if len(info) >= 3:
            row[4] = info[2]
        if len(info) >= 4:
            row[5] = info[3]
        if len(info) >= 5:
            row[6] = info[4]
        if len(info) >= 6:
            row[7] = info[5]
        if len(info) >= 7:
            row[8] = info[6]
        if len(info) >= 8:
            row[9] = info[7]
        if len(info) >= 9:
            row[10] = info[8]
        if len(info) >= 10:
            row[11] = info[9]
        results.add_row(row)
    print(results)